### Extract weights, biases, inputs, outputs of VGG16 into a pickled dictionary

Dictionary format:

```
* arrays_dict
    * 'layer_name':
        * 'input ':
        * 'output':
        * 'kernel': - if conv in layer_name
        * 'bias  ': - if conv in layer_name
```

In [ ]:
import tensorflow as tf
import numpy as np
import pickle

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

tf.keras.backend.set_floatx('float16')

In [ ]:
vgg16_model = tf.keras.applications.VGG16(include_top=True, weights='imagenet')

In [ ]:
img = image.load_img('elephant.jpg', target_size=(224, 224))
x   = image.img_to_array(img)
x   = np.expand_dims(x, axis=0)
x   = preprocess_input(x)

out = vgg16_model.predict(x)

In [ ]:
layer_names = [layer.name for layer in vgg16_model.layers]
layer_names

In [ ]:
arrays_dict = {}
config = {}
indices  = {}
tensors = []
input_tensor = None
i = 0

for name in layer_names:
    if 'input' in name:
        input_tensor = vgg16_model.get_layer(name).output
    else:
        layer = vgg16_model.get_layer(name)
        
        arrays_dict[name] = {}
        config[name] = {}
        config[name]['layer'] = layer
        
        if 'conv' in name:
            arrays_dict[name]['kernel'] = layer.weights[0].numpy()
            arrays_dict[name]['bias']   = layer.weights[1].numpy()
        
        tensors += [layer.input]
        config[name]['input_index']  = i
        i += 1
        
        tensors += [layer.output]
        config[name]['output_index'] = i
        i += 1
        
tmp_model = tf.keras.Model(inputs=input_tensor, outputs=tensors)
arrays = tmp_model.predict(x)

assert len(arrays) == i

for layer_name in arrays_dict:
    arrays_dict[layer_name]['input']  = arrays[config[layer_name]['input_index']]
    arrays_dict[layer_name]['output'] = arrays[config[layer_name]['output_index']]

In [ ]:
arrays_dict['block1_conv1']['bias']

In [ ]:
with open('vgg16_dict.pickle', 'wb') as f:
    pickle.dump(arrays_dict, f)

In [ ]:
arrays_dict_open = pickle.load(open('vgg16_dict.pickle', 'rb'))
arrays_dict_open['block3_conv2']['input'].shape